In [1]:
import numpy as np
import pandas as pd

In [2]:
batting_df = pd.read_csv("db/batting4.csv")

In [ ]:
batting_df.info()

In [3]:
X = batting_df[["R","RBI","SB","AVG","SLG","Games_C","Games_2B","Games_3B","Games_SS"]]
y = batting_df["HOF_member"]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [11]:
# split data into test and train
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=27)


In [12]:
# Normalize the data.  Here, we are using StandardScaler
X_scaler = StandardScaler().fit(X_train)
#X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



C:\Users\Laptop 1\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Laptop 1\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """
C:\Users\Laptop 1\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [7]:
model = LogisticRegression(C=1.0, penalty='l2', random_state=0, solver="saga", max_iter=5000)

In [ ]:
X_train_scaled.shape

In [13]:
model.fit(X_train_scaled,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=5000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

In [14]:
from sklearn.metrics import mean_squared_error

In [15]:
predictions = model.predict(X_test_scaled)
MSE = mean_squared_error(y_test, predictions)
r2 = model.score(X_test_scaled, y_test)

print(f"MSE: {MSE}, R2: {r2}") 


MSE: 0.06936416184971098, R2: 0.930635838150289


If curious about which players in the test split were predicted for what, we can take these two outputs, import and pair them

In [ ]:
X_test["predictions"] = predictions
X_test.head()

In [ ]:
X_test.to_csv("db/Xtest.csv", index=False, header=True)

Here's what we've been waiting for:  Let's make Hall-Of-Fame predictions for more recent and current players.

In [16]:
recent_df = pd.read_csv("db/batting_recent.csv")

In [17]:
recent_players = recent_df["playerID"]
X_recent = recent_df[["R","RBI","SB","AVG","SLG","Games_C","Games_2B","Games_3B","Games_SS"]]

In [18]:
X_recent_scaled = X_scaler.transform(X_recent)

C:\Users\Laptop 1\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [19]:
predictions_recent = model.predict(X_recent_scaled)

In [20]:
X_recent["predictions"] = predictions_recent
X_recent["playerID"] = recent_players

C:\Users\Laptop 1\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
people_df = pd.read_csv("db/people.csv")

In [22]:
players_df = people_df[["playerID","nameFirst","nameLast"]]
players_df["nameFull"] = players_df["nameFirst"]+' '+players_df["nameLast"]
players_df.head()

C:\Users\Laptop 1\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,playerID,nameFirst,nameLast,nameFull
0,aardsda01,David,Aardsma,David Aardsma
1,aaronha01,Hank,Aaron,Hank Aaron
2,aaronto01,Tommie,Aaron,Tommie Aaron
3,aasedo01,Don,Aase,Don Aase
4,abadan01,Andy,Abad,Andy Abad


In [23]:
X_recent.to_csv("db/Xrecent.csv", index=False, header=True)

PermissionError: [Errno 13] Permission denied: 'db/Xrecent.csv'

In [25]:
Predicted_for_HOF = X_recent.loc[X_recent["predictions"] == 0, :]
Predicted_for_HOF = pd.merge(Predicted_for_HOF, players_df, on="playerID", how="left")
del Predicted_for_HOF["nameFirst"]
del Predicted_for_HOF["nameLast"]
display(Predicted_for_HOF)

,R,RBI,SB,AVG,SLG,Games_C,Games_2B,Games_3B,Games_SS,predictions,playerID,nameFull
0,801,557,271,0.275119,0.370828,0.0,0.0,0.0,1453.0,0,andruel01,Elvis Andrus
1,601,449,143,0.273636,0.373803,0.0,52.0,13.0,1219.0,0,aybarer01,Erick Aybar
2,699,678,49,0.258625,0.494331,0.0,5.0,378.0,0.0,0,bautijo02,Jose Bautista
3,722,740,92,0.265957,0.479855,0.0,0.0,0.0,0.0,0,bayja01,Jason Bay
4,700,785,90,0.271402,0.459470,0.0,1.0,1414.0,7.0,0,beltrad01,Adrian Beltre
5,632,609,33,0.264357,0.444166,0.0,1.0,894.0,1.0,0,blakeca01,Casey Blake
6,676,361,340,0.266122,0.357621,0.0,0.0,0.0,0.0,0,bournmi01,Michael Bourn
7,509,453,82,0.274627,0.448060,0.0,0.0,0.0,0.0,0,bradlmi01,Milton Bradley
8,528,510,113,0.294826,0.433402,0.0,0.0,0.0,0.0,0,brantmi02,Michael Brantley
9,751,838,62,0.248607,0.471662,0.0,0.0,0.0,0.0,0,bruceja01,Jay Bruce
